In [2]:
import adept_envs
import gym
import glob
import os
from parse_mjl import parse_mjl_logs
import numpy as np

In [3]:
env = gym.make("kitchen_relax-v1")

Reading configurations for Franka
Initializing Franka sim
registering the liscence


/home/sholto/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
obs = env.reset()


In [5]:
demo_dir = os.getcwd() + '/' + "relay-policy-learning/kitchen_demos_multitask/"

demos = glob.glob(demo_dir + "/**/*.mjl", recursive=True)
print([d.strip(demo_dir) for d in demos])

['b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_06_46.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_05_26.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_05_06.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_02_41.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_04_03.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_04_24.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_01_21.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_02_01.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_01_41.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_03_42.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_07_06.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_03_01.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_04_45.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_07_44.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_02_21.mj', 'b_bottomknob_hinge/kitchen_playdata_2019_07_11_18_06_06.mj', 'b_bott

In [6]:
file = demos[1]
skip = 10
data = parse_mjl_logs(file, skip)
data

{'nq': 30,
 'nv': 29,
 'nu': 9,
 'nmocap': 1,
 'nsensordata': 0,
 'name': b'franka_mocap_studyTable_buttons',
 'time': array([ 2.19400001,  2.21399999,  2.23399997,  2.25399995,  2.27399993,
         2.29399991,  2.31399989,  2.33400011,  2.35400009,  2.37400007,
         2.39400005,  2.41400003,  2.43400002,  2.454     ,  2.47399998,
         2.49399996,  2.51399994,  2.53399992,  2.5539999 ,  2.57399988,
         2.5940001 ,  2.61400008,  2.63400006,  2.65400004,  2.67400002,
         2.69400001,  2.71399999,  2.73399997,  2.75399995,  2.77399993,
         2.79399991,  2.81399989,  2.83400011,  2.85400009,  2.87400007,
         2.89400005,  2.91400003,  2.93400002,  2.954     ,  2.97399998,
         2.99399996,  3.01399994,  3.03399992,  3.0539999 ,  3.07399988,
         3.0940001 ,  3.11400008,  3.13400006,  3.15400004,  3.17400002,
         3.19400001,  3.21399999,  3.23399997,  3.25399995,  3.27399993,
         3.29399991,  3.31399989,  3.33400011,  3.35400009,  3.37400007,
      

In [7]:
print(data['ctrl'].shape, data['qpos'].shape, data['qvel'].shape)

(908, 9) (908, 30) (908, 29)


In [25]:
file = demos[np.random.choice(len(demos))]
skip = 30 # this is frame skip
data = parse_mjl_logs(file, skip)
env.reset()
for i in range(0,len(data['qpos'])):
    env.robot.reset(env, data['qpos'][i], data['qvel'][i])
    env.render()

# The issue!
Now, the problem is that cntrl is at the full time frequency of the env - which is really fine. That means if frame skip is anything but 1 then the ctrl labels are wrong. Ideally we want a more reasonable time freq, which means we need to take the difference between the first 9 elements of q_pos, divide it by the time between and then that gives us our desired vel controls. Even better, tonight I'll modify the env for position control. 

See, the below fails - 

In [29]:
file = demos[np.random.choice(len(demos))]
skip = 1 # this is frame skip
data = parse_mjl_logs(file, skip)
env.reset()
env.robot.reset(env, data['qpos'][0], data['qvel'][0])
for i in range(0,len(data['qpos'])):
    
    env.step(data['ctrl'][i]) # the key change
    env.render()

# Env Info 

## Action Space

Action space is 9 dimensional, commanded via velocity. In franka robot.py, the function cntrl_velocity_limits bounds the desired velocity step, then returns a next desired positon (prevpos + cntrl_vel*step_size), so really rather than absolute position it is commanding desired pos, using a relative offset commanded via velocity assuming a constant timestep (0.08). This means it'd be easy to adapt to commanding via relative positon, or velocity. 

## Obs Space

60 Dimensional. 9 for self, 21 for obs, 30 for goal. Self is unused in reward computation, only the following indices are 



In [ ]:
OBS_ELEMENT_INDICES = {
    'bottom burner': np.array([11, 12]),
    'top burner': np.array([15, 16]),
    'light switch': np.array([17, 18]),
    'slide cabinet': np.array([19]),
    'hinge cabinet': np.array([20, 21]),
    'microwave': np.array([22]),
    'kettle': np.array([23, 24, 25, 26, 27, 28, 29]),
    }

We can reset it to any qpos,qvel self.robot.reset(self, reset_pos, reset_vel) (sizes 30, 29). qvel is actually returned as 30 dimensional internally (but never used so its not an issue. This off by one issue comes from the sim not returning the vel of one of the object elements I think, can't find out which one though. Not a big issue, vel isn't part of the obs state so clearly not that key.